In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.5 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=6e2a7daa3858a0085579023cfd4db48a55a9e59221f051ce6d00bf5df1c42d93
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [4]:
!git clone https://github.com/feralvam/easse.git && cd easse && pip install .

Cloning into 'easse'...
remote: Enumerating objects: 1964, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 1964 (delta 109), reused 96 (delta 96), pack-reused 1827
Receiving objects: 100% (1964/1964), 33.15 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (1231/1231), done.
Processing /content/easse
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-jnie1q15/tseval_11f36a10a2b64610afe3f37a7bb6a9e9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-jnie1q15/tseval_11f36a10a2b64610afe3f37a7bb6a9e9
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit dea8863683ea5946fd50184883c9be7a7339e821
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local working folder
py_vncorenlp.download_model(save_dir='/content')
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

In [7]:
def text_segment(text):
  return " ".join(rdrsegmenter.word_segment(text))

In [8]:
import json

def load_json(file_path):
  with open(file_path, 'r', encoding='utf-8') as json_file:
    content = json.load(json_file)
  return content

def save_json(file_path, data):
  with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=2)

# Bert Score

In [9]:
from typing import List
from bert_score import BERTScorer

scorer = BERTScorer(model_type="vinai/phobert-large", num_layers=17, batch_size=1, device="cuda")
def get_bertscore_sentence_scores1(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    lowercase: bool = False,
    tokenizer: str = "13a",
):


    # sys_sents = [utils_prep.normalize(sent, lowercase, tokenizer) for sent in sys_sents]
    # refs_sents = [[utils_prep.normalize(sent, lowercase, tokenizer) for sent in ref_sents] for ref_sents in refs_sents]
    refs_sents = [list(r) for r in zip(*refs_sents)]

    return scorer.score(sys_sents, refs_sents)

def corpus_bertscore1(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    lowercase: bool = False,
    tokenizer: str = "13a",
):
    all_scores = get_bertscore_sentence_scores1(sys_sents, refs_sents, lowercase, tokenizer)
    avg_scores = [s.mean(dim=0) for s in all_scores]
    precision = avg_scores[0].cpu().item()
    recall = avg_scores[1].cpu().item()
    f1 = avg_scores[2].cpu().item()
    return precision, recall, f1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

In [10]:
sys_sents = [text_segment(x) for x in ["Tôi ăn táo táo táo táo."]]
refs_sents = [[text_segment("Bakhtin nói trong cuốn sách về Đôxtôiepxki rằng, sự thật không tồn tại trong đầu một người, mà nó xuất hiện khi mọi người cùng nhau tìm kiếm và trò chuyện.")]]

corpus_bertscore1(sys_sents, refs_sents)

(0.35290294885635376, 0.2821926772594452, 0.3136114478111267)

# Corpus quality estimation

In [11]:
import Levenshtein
from collections import Counter

def safe_division(a, b):
    if b == 0:
        return b
    return a / b

def count_characters(sentence):
    return len(sentence)

def count_sentences(sentence):
    n_sentence = sentence.replace("_", "")
    return len(rdrsegmenter.word_segment(n_sentence))

def to_words(sentence):
    return sentence.split()

def remove_punctuation_tokens(word_list):
    punctuation_list = [".", ",", ";", "?", "!", "(", ")", "\"", "-", "_", ":"]
    return [x for x in word_list if x not in punctuation_list]

def count_words(sentence, tokenize=True, remove_punctuation=False):
    return len(to_words(sentence))

def get_compression_ratio(complex_sentence, simple_sentence):
    return safe_division(count_characters(simple_sentence), count_characters(complex_sentence))

def count_sentence_splits(complex_sentence, simple_sentence):
    return safe_division(count_sentences(simple_sentence), count_sentences(complex_sentence))

def is_exact_match(complex_sentence, simple_sentence):
    return complex_sentence == simple_sentence

def get_additions_proportion(complex_sentence, simple_sentence):
    n_additions = sum((Counter(to_words(simple_sentence)) - Counter(to_words(complex_sentence))).values())
    return n_additions / max(count_words(complex_sentence), count_words(simple_sentence))

def get_deletions_proportion(complex_sentence, simple_sentence):
    n_deletions = sum((Counter(to_words(complex_sentence)) - Counter(to_words(simple_sentence))).values())
    return n_deletions / max(count_words(complex_sentence), count_words(simple_sentence))

def get_levenshtein_similarity(complex_sentence, simple_sentence):
    return Levenshtein.ratio(complex_sentence, simple_sentence)

In [12]:
def get_average(vectorizer, orig_sentences, sys_sentences):
    cumsum = 0
    count = 0
    for orig_sentence, sys_sentence in zip(orig_sentences, sys_sentences):
        cumsum += vectorizer(orig_sentence, sys_sentence)
        count += 1
    return cumsum / count

def corpus_quality_estimation1(
    orig_sentences: List[str], sys_sentences: List[str], lowercase: bool = False, tokenizer: str = '13a'
):
    # orig_sentences = [normalize(sent, lowercase, tokenizer) for sent in orig_sentences]
    # sys_sentences = [normalize(sent, lowercase, tokenizer) for sent in sys_sentences]
    return {
        'Compression ratio': get_average(get_compression_ratio, orig_sentences, sys_sentences),
        'Sentence splits': get_average(count_sentence_splits, orig_sentences, sys_sentences),
        'Levenshtein similarity': get_average(get_levenshtein_similarity, orig_sentences, sys_sentences),
        'Exact copies': get_average(is_exact_match, orig_sentences, sys_sentences),
        'Additions proportion': get_average(get_additions_proportion, orig_sentences, sys_sentences),
        'Deletions proportion': get_average(get_deletions_proportion, orig_sentences, sys_sentences),
        # 'Lexical complexity score': get_average(
        #     wrap_single_sentence_vectorizer(get_wordrank_score), orig_sentences, sys_sentences
        # ),
    }

In [13]:
orig_sentences =  [text_segment(x) for x in ["Trong cuốn Những vấn đề thi pháp Đôxtôiepxki , Bakhtin nhận xét : \" Chân lí không nảy sinh và không nằm trong đầu một con người riêng lẻ , nó nảy sinh giữa những con người đang cùng nhau đi tìm chân lý và trong quá trình giao tiếp đối thoại với nhau \" ."]]
sys_sentences = [text_segment(x) for x in ["Trong cuốn \" Những vấn đề thi pháp Đôxtôiepxki \", Bakhtin nói rằng lý lẽ không chỉ do một người mà còn do nhiều người cùng tìm kiếm sự thật và giao tiếp với nhau ."]]

print(orig_sentences)
print(sys_sentences)
corpus_quality_estimation1(
    [text_segment(x) for x in ["Trong cuốn Những vấn đề thi pháp Đôxtôiepxki , Bakhtin nhận xét : \" Chân lí không nảy sinh và không nằm trong đầu một con người riêng lẻ , nó nảy sinh giữa những con người đang cùng nhau đi tìm chân lý và trong quá trình giao tiếp đối thoại với nhau \" ."]],
    [text_segment(x) for x in ["Trong cuốn \" Những vấn đề thi pháp Đôxtôiepxki \", Bakhtin nói rằng lý lẽ không chỉ do một người mà còn do nhiều người cùng tìm kiếm sự thật và giao tiếp với nhau ."]]
)

['Trong cuốn Những vấn_đề thi_pháp Đôxtôiepxki , Bakhtin nhận_xét : " Chân_lí không nảy_sinh và không nằm trong đầu một con_người riêng_lẻ , nó nảy_sinh giữa những con_người đang cùng nhau đi tìm chân_lý và trong quá_trình giao_tiếp đối_thoại với nhau " .']
['Trong cuốn " Những vấn_đề thi_pháp Đôxtôiepxki " , Bakhtin nói rằng lý_lẽ không_chỉ do một người mà_còn do nhiều người cùng tìm_kiếm sự_thật và giao_tiếp với nhau .']


{'Compression ratio': 0.6482213438735178,
 'Sentence splits': 1.0,
 'Levenshtein similarity': 0.6139088729016786,
 'Exact copies': 0.0,
 'Additions proportion': 0.27906976744186046,
 'Deletions proportion': 0.6046511627906976}

# SARI

In [14]:
from collections import Counter
from typing import List
import easse.utils.preprocessing as utils_prep

NGRAM_ORDER = 4


def compute_precision_recall(sys_correct, sys_total, ref_total):
    precision = 0.0
    if sys_total > 0:
        precision = sys_correct / sys_total

    recall = 0.0
    if ref_total > 0:
        recall = sys_correct / ref_total

    return precision, recall


def compute_f1(precision, recall):
    f1 = 0.0
    if precision > 0 or recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    return f1


def compute_micro_sari(
    add_hyp_correct,
    add_hyp_total,
    add_ref_total,
    keep_hyp_correct,
    keep_hyp_total,
    keep_ref_total,
    del_hyp_correct,
    del_hyp_total,
    del_ref_total,
    use_f1_for_deletion=True,
):
    """
    This is the version described in the original paper. Follows the equations.
    """
    add_precision = [0] * NGRAM_ORDER
    add_recall = [0] * NGRAM_ORDER
    keep_precision = [0] * NGRAM_ORDER
    keep_recall = [0] * NGRAM_ORDER
    del_precision = [0] * NGRAM_ORDER
    del_recall = [0] * NGRAM_ORDER

    for n in range(NGRAM_ORDER):
        add_precision[n], add_recall[n] = compute_precision_recall(
            add_hyp_correct[n], add_hyp_total[n], add_ref_total[n]
        )
        keep_precision[n], keep_recall[n] = compute_precision_recall(
            keep_hyp_correct[n], keep_hyp_total[n], keep_ref_total[n]
        )
        del_precision[n], del_recall[n] = compute_precision_recall(
            del_hyp_correct[n], del_hyp_total[n], del_ref_total[n]
        )

    avg_add_precision = sum(add_precision) / NGRAM_ORDER
    avg_add_recall = sum(add_recall) / NGRAM_ORDER
    avg_keep_precision = sum(keep_precision) / NGRAM_ORDER
    avg_keep_recall = sum(keep_recall) / NGRAM_ORDER
    avg_del_precision = sum(del_precision) / NGRAM_ORDER
    avg_del_recall = sum(del_recall) / NGRAM_ORDER

    add_f1 = compute_f1(avg_add_precision, avg_add_recall)

    keep_f1 = compute_f1(avg_keep_precision, avg_keep_recall)

    if use_f1_for_deletion:
        del_score = compute_f1(avg_del_precision, avg_del_recall)
    else:
        del_score = avg_del_precision

    return add_f1, keep_f1, del_score


def extract_ngrams(line, min_order=1, max_order=NGRAM_ORDER) -> List[Counter]:
    ngrams_per_order = []
    tokens = line.split()
    for n in range(min_order, max_order + 1):
        ngrams = Counter()
        for i in range(0, len(tokens) - n + 1):
            ngram = " ".join(tokens[i : i + n])
            ngrams[ngram] += 1
        ngrams_per_order.append(ngrams)

    return ngrams_per_order


def multiply_counter(c, v):
    c_aux = Counter()
    for k in c.keys():
        c_aux[k] = c[k] * v

    return c_aux


def compute_ngram_stats(orig_sents: List[str], sys_sents: List[str], refs_sents: List[List[str]]):
    """
    Input:
    orig_sents: list of original sentences (len = n_samples)
    sys_sents: list of system sentences (len = n_samples)
    refs_sents: list of list of reference sentences (shape = (n_references, n_samples))
    """
    assert len(orig_sents) == len(
        sys_sents
    ), "Original sentences and system sentences don't have the same number of samples"
    assert all(
        len(ref_sents) == len(orig_sents) for ref_sents in refs_sents
    ), "Reference sentences don't have the shape (n_references, n_samples)"
    add_sys_correct = [0] * NGRAM_ORDER
    add_sys_total = [0] * NGRAM_ORDER
    add_ref_total = [0] * NGRAM_ORDER

    keep_sys_correct = [0] * NGRAM_ORDER
    keep_sys_total = [0] * NGRAM_ORDER
    keep_ref_total = [0] * NGRAM_ORDER

    del_sys_correct = [0] * NGRAM_ORDER
    del_sys_total = [0] * NGRAM_ORDER
    del_ref_total = [0] * NGRAM_ORDER

    for orig_sent, sys_sent, *ref_sents in zip(orig_sents, sys_sents, *refs_sents):
        orig_ngrams = extract_ngrams(orig_sent)
        sys_ngrams = extract_ngrams(sys_sent)

        refs_ngrams = [Counter() for _ in range(NGRAM_ORDER)]
        for ref_sent in ref_sents:
            ref_ngrams = extract_ngrams(ref_sent)
            for n in range(NGRAM_ORDER):
                refs_ngrams[n] += ref_ngrams[n]

        num_refs = len(ref_sents)
        for n in range(NGRAM_ORDER):
            # ADD
            # added by the hypothesis (binary)
            sys_and_not_orig = set(sys_ngrams[n]) - set(orig_ngrams[n])
            add_sys_total[n] += len(sys_and_not_orig)
            # added by the references (binary)
            ref_and_not_orig = set(refs_ngrams[n]) - set(orig_ngrams[n])
            add_ref_total[n] += len(ref_and_not_orig)
            # added correctly (binary)
            add_sys_correct[n] += len(sys_and_not_orig & set(refs_ngrams[n]))

            # KEEP
            # kept by the hypothesis (weighted)
            orig_and_sys = multiply_counter(orig_ngrams[n], num_refs) & multiply_counter(sys_ngrams[n], num_refs)
            keep_sys_total[n] += sum(orig_and_sys.values())
            # kept by the references (weighted)
            orig_and_ref = multiply_counter(orig_ngrams[n], num_refs) & refs_ngrams[n]
            keep_ref_total[n] += sum(orig_and_ref.values())
            # kept correctly?
            keep_sys_correct[n] += sum((orig_and_sys & orig_and_ref).values())

            # DELETE
            # deleted by the hypothesis (weighted)
            orig_and_not_sys = multiply_counter(orig_ngrams[n], num_refs) - multiply_counter(sys_ngrams[n], num_refs)
            del_sys_total[n] += sum(orig_and_not_sys.values())
            # deleted by the references (weighted)
            orig_and_not_ref = multiply_counter(orig_ngrams[n], num_refs) - refs_ngrams[n]
            del_ref_total[n] += sum(orig_and_not_ref.values())
            # deleted correctly
            del_sys_correct[n] += sum((orig_and_not_sys & orig_and_not_ref).values())

    return (
        add_sys_correct,
        add_sys_total,
        add_ref_total,
        keep_sys_correct,
        keep_sys_total,
        keep_ref_total,
        del_sys_correct,
        del_sys_total,
        del_ref_total,
    )


def compute_precision_recall_f1(sys_correct, sys_total, ref_total):
    precision = 0.0
    if sys_total > 0:
        precision = sys_correct / sys_total

    recall = 0.0
    if ref_total > 0:
        recall = sys_correct / ref_total

    f1 = 0.0
    if precision > 0 and recall > 0:
        f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1


def compute_macro_sari(
    add_sys_correct,
    add_sys_total,
    add_ref_total,
    keep_sys_correct,
    keep_sys_total,
    keep_ref_total,
    del_sys_correct,
    del_sys_total,
    del_ref_total,
    use_f1_for_deletion=True,
):
    """
    This is the version released as a JAVA implementation and which was used in their experiments,
    as stated by the authors: https://github.com/cocoxu/simplification/issues/8
    """
    add_f1 = 0.0
    keep_f1 = 0.0
    del_f1 = 0.0
    for n in range(NGRAM_ORDER):
        _, _, add_f1_ngram = compute_precision_recall_f1(add_sys_correct[n], add_sys_total[n], add_ref_total[n])
        _, _, keep_f1_ngram = compute_precision_recall_f1(keep_sys_correct[n], keep_sys_total[n], keep_ref_total[n])
        if use_f1_for_deletion:
            _, _, del_score_ngram = compute_precision_recall_f1(del_sys_correct[n], del_sys_total[n], del_ref_total[n])
        else:
            del_score_ngram, _, _ = compute_precision_recall_f1(del_sys_correct[n], del_sys_total[n], del_ref_total[n])
        add_f1 += add_f1_ngram / NGRAM_ORDER
        keep_f1 += keep_f1_ngram / NGRAM_ORDER
        del_f1 += del_score_ngram / NGRAM_ORDER
    return add_f1, keep_f1, del_f1


def get_corpus_sari_operation_scores(
    orig_sents: List[str],
    sys_sents: List[str],
    refs_sents: List[List[str]],
    lowercase: bool = True,
    tokenizer: str = '13a',
    legacy=False,
    use_f1_for_deletion=True,
    use_paper_version=False,
):
    """
    Inputs:
    orig_sents: list of original sentences (len = n_samples)
    sys_sents: list of system sentences (len = n_samples)
    refs_sents: list of list of reference sentences (shape = (n_references, n_samples))
    legacy: Allows reproducing scores reported in previous work.
    It replicates a bug in the original JAVA implementation where only the system outputs and the reference sentences
    are further tokenized.
    In addition, it assumes that all sentences are already lowercased.
    """
    # if legacy:
    #     lowercase = False
    # else:
    #     orig_sents = [utils_prep.normalize(sent, lowercase, tokenizer) for sent in orig_sents]

    # sys_sents = [utils_prep.normalize(sent, lowercase, tokenizer) for sent in sys_sents]
    # refs_sents = [[utils_prep.normalize(sent, lowercase, tokenizer) for sent in ref_sents] for ref_sents in refs_sents]

    stats = compute_ngram_stats(orig_sents, sys_sents, refs_sents)

    if not use_paper_version:
        add_score, keep_score, del_score = compute_macro_sari(*stats, use_f1_for_deletion=use_f1_for_deletion)
    else:
        add_score, keep_score, del_score = compute_micro_sari(*stats, use_f1_for_deletion=use_f1_for_deletion)
    return 100.0 * add_score, 100.0 * keep_score, 100.0 * del_score


def corpus_sari1(*args, **kwargs):
    add_score, keep_score, del_score = get_corpus_sari_operation_scores(*args, **kwargs)
    return (add_score + keep_score + del_score) / 3


In [15]:
# from easse.sari import corpus_sari

def sentence_sari(orig_sent: str, sys_sent: str, ref_sents:list[str]):
  return corpus_sari1(orig_sents=[orig_sent],
                sys_sents=[sys_sent],
                refs_sents=[ref_sents], legacy=True)
orig_sent = text_segment("Trước đây, việc nghiên cứu văn học Việt Nam so với nước ngoài thường thấy tự ti về văn học dân tộc. Vì vậy, người ta chủ yếu hỏi \" văn học nước ngoài ảnh hưởng đến Việt Nam như thế nào \", chứ ít hỏi \" việc chúng ta chọn lọc văn học nước ngoài như thế nào và tại sao \" .")
sys_sents = text_segment("Trước đây, khi nghiên cứu văn học Việt Nam so với nước ngoài, người ta thường thấy tự ti về văn học dân tộc. Vì thế, họ chủ yếu khám phá cách văn học nước ngoài ảnh hưởng đến Việt Nam, ít xem xét cách Việt Nam chọn lựa và tiếp thu văn học nước ngoài, hoặc tại sao .")
refs_sents = [text_segment("Trước đây, khi nghiên cứu văn học Việt Nam và văn học nước ngoài, người ta thường tự ti về văn học của mình. Họ chủ yếu xem xét văn học nước ngoài ảnh hưởng đến văn học Việt Nam như thế nào mà ít nói đến việc chúng ta đã chọn và học hỏi từ văn học nước ngoài ra sao.")]

sentence_sari(orig_sent, sys_sents, refs_sents)

52.063492676434265

# BLEU

In [16]:
"""
Implements the evaluation metrics based on BLEU score
"""

import numpy as np
from typing import List

from sacrebleu.metrics import BLEU

import easse.utils.preprocessing as utils_prep


def corpus_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = "exp",
    smooth_value: float = None,
    force: bool = True,
    lowercase: bool = False,
    tokenizer: str = "13a",
    effective_order: bool = False,
):
    # sys_sents = [utils_prep.normalize(sent, lowercase, tokenizer) for sent in sys_sents]
    # refs_sents = [[utils_prep.normalize(sent, lowercase, tokenizer) for sent in ref_sents] for ref_sents in refs_sents]

    bleu_scorer = BLEU(lowercase=False, force=force, tokenize="none", smooth_method=smooth_method, smooth_value=smooth_value, effective_order=effective_order)

    return bleu_scorer.corpus_score(
        sys_sents,
        refs_sents,
    ).score


def sentence_bleu1(
    sys_sent: str,
    ref_sents: List[str],
    smooth_method: str = "floor",
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = "13a",
    effective_order: bool = True,
):

    return corpus_bleu(
        [sys_sent],
        [[ref] for ref in ref_sents],
        smooth_method,
        smooth_value,
        force=True,
        lowercase=lowercase,
        tokenizer=tokenizer,
        effective_order=effective_order,
    )


def corpus_averaged_sentence_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = "floor",
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = "13a",
    effective_order: bool = True,
):

    scores = []
    for sys_sent, *ref_sents in zip(sys_sents, *refs_sents):
        scores.append(
            sentence_bleu1(
                sys_sent,
                ref_sents,
                smooth_method,
                smooth_value,
                lowercase=lowercase,
                tokenizer=tokenizer,
                effective_order=effective_order,
            )
        )
    return np.mean(scores)


In [17]:
# from easse.bleu import sentence_bleu

sentence_bleu1("Tôi học lớp 5 .", ["Tôi là học_sinh lớp 5 ."])

23.394743548827705

In [18]:
# bart = load_json("/content/bart_2epochs.json")
# gemsura_7b_216 = load_json("/content/result-gemsura7b-checkpoint216.json")["result"]
# gemsura_7b_252 = load_json("/content/result-gemsura7b-checkpoint252.json")["result"]
# gemsura_7b_288 = load_json("/content/result-gemsura7b-checkpoint288.json")["result"]
# vistral_7b_144 = load_json("/content/result-vistral7b-chat-checkpoint144.json")["result"]
# gemsura_2b = load_json("/content/ss-result-qlora-gemsura2b-a128r256.json")["result"]
# phogpt4b = load_json("/content/ss-result-qlora-phogpt4b-a256r512-v2.json")["result"]
# phogpt4b_chat = load_json("/content/ss-result-qlora-phogpt4b-chat-lr1e-4-attn-ffn-updown.json")

In [19]:
# bart = [{"sentence": x["sentence"].replace("_", " "), "gpt4": x["result"].replace("_", " "), "prediction": x["pred"].replace("_", " ")} for x in bart]
# gemsura_7b_216 = [{"sentence": x["sentence"], "gpt4": x["simple"], "prediction": x["qlora-gemsura2b"]} for x in gemsura_7b_216]
# gemsura_7b_252 = [{"sentence": x["sentence"], "gpt4": x["simple"], "prediction": x["qlora-phogpt4b-chat"]} for x in gemsura_7b_252]
# gemsura_7b_288 = [{"sentence": x["sentence"], "gpt4": x["simple"], "prediction": x["qlora-gemsura2b"]} for x in gemsura_7b_288]
# vistral_7b_144 = [{"sentence": x["sentence"], "gpt4": x["simple"], "prediction": x["vistral7b-chat"]} for x in vistral_7b_144]
# gemsura_2b = [{"sentence": x["sentence"], "gpt4": x["simple"], "prediction": x["qlora-gemsura2b"]} for x in gemsura_2b]
# phogpt4b = [{"sentence": x["sentence"], "gpt4": x["simple"], "prediction": x["qlora-phogpt4b-chat"]} for x in phogpt4b]
# phogpt4b_chat = [{"sentence": x["sentence"], "gpt4": x["simple"], "prediction": x["qlora-phogpt4b-chat"]} for x in phogpt4b_chat]

In [20]:
# phogpt4b_chat[0]

# Evaluation

In [21]:
path_eval = '/content/drive/MyDrive/Sentence Simplification/colab-22/eval.json'
path_train = '/content/drive/MyDrive/Sentence Simplification/colab-22/train.json'
path_res_llama_1 = '/content/drive/MyDrive/Sentence Simplification/colab-22/ss-llama3-eps3-lr2e-05-constant-r8a16dr0.5-ck286.json'

In [22]:
data = load_json(path_res_llama_1)["result"]

In [23]:
data[0]

{'sentence': 'Ở phần sau của bài viết , chúng tôi sẽ đi vào trình bày lại một số thực hành để có thể quan sát rõ hơn cách đặt vấn đề và các thao tác trong nghiên cứu .',
 'result': 'Trong phần tiếp theo, chúng tôi sẽ giới thiệu một số thực hành giúp hiểu rõ hơn về cách đặt vấn đề và thao tác nghiên cứu.',
 'gemsura2b': 'Trong phần sau, chúng tôi sẽ nói về một số cách thực hành để hiểu rõ hơn về cách đặt vấn đề và thao tác trong nghiên cứu.'}

In [24]:
eval = load_json(path_eval)
train = load_json(path_train)

In [25]:
result = [text_segment(x["sentence"]) + " " + text_segment(x["result"]) for x in eval]

In [26]:
temp = [x for x in result if len(x.split(" ")) > 256]

In [27]:
len(temp[0].split())

354

In [28]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [29]:
# simCSE similarity

def compute_sim(s1, s2):
  sentences = [s1, s2]
  sentences = [text_segment(sentence) for sentence in sentences]
  embeddings = model.encode(sentences)

  return cos_sim(embeddings[0], embeddings[1])

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

def cos_sim(a, b):
  return cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0][0]

In [31]:
def compute_metrics(data):
    word_sari_score = 0
    syllable_sari_score = 0
    bleu_score = 0
    sim_score = 0
    segmented_data = [
        {
            "sentence": text_segment(x["sentence"]),
            "result": text_segment(x["result"]),
            "pred": text_segment(x["gemsura2b"])
        } for x in data if len((text_segment(x["sentence"]) + " " + text_segment(x["result"])).split(" ")) <= 256
    ]
    for i in segmented_data:
      pred = i["pred"]
      input = i["sentence"]
      label = i["result"]
      word_sari_score += sentence_sari(input, pred, [label])
      bleu_score += sentence_bleu1(pred, [label])
      sim_score += compute_sim(input, label)
    sim_score /= len(segmented_data)
    word_sari_score /= len(data)
    bleu_score /= len(data)
    quality_dict = corpus_quality_estimation1(
        [x["sentence"] for x in segmented_data],
         [x["pred"] for x in segmented_data]
    )
    precision, recall, f1 = corpus_bertscore1([x["pred"] for x in segmented_data], [[x["result"]] for x in segmented_data])
    return {
        "SARI_word": word_sari_score,
        "SARI_syllable": syllable_sari_score,
        "BLEU": bleu_score,
        "Corpus quality estimation": quality_dict,
        "sim_score": sim_score,
        "bertscore": {
            "precision": precision,
            "recall": recall,
            "f1": f1
        }
    }

In [32]:
def compute_metrics_for_gpt4(data):
    segmented_data = [
        {
            "sentence": text_segment(x["sentence"]),
            "result": text_segment(x["result"]),
        } for x in data if len((text_segment(x["sentence"]) + " " + text_segment(x["result"])).split(" ")) <= 256
    ]
    sim_score = 0
    print(len(segmented_data))
    for i in segmented_data:
      input = i["sentence"]
      label = i["result"]
      sim_score += compute_sim(input, label)
    sim_score /= len(segmented_data)
    quality_dict = corpus_quality_estimation1(
        [x["sentence"] for x in segmented_data],
         [x["result"] for x in segmented_data]
    )

    # precision, recall, f1 = corpus_bertscore1([x["result"] for x in segmented_data], [[x["sentence"]] for x in segmented_data])
    return {
        "Corpus quality estimation": quality_dict,
        "sim_score": sim_score,
        # "bertscore": {
        #     "precision": precision,
        #     "recall": recall,
        #     "f1": f1
        # }
    }

In [33]:
compute_metrics(data)

{'SARI_word': 51.562334826565234,
 'SARI_syllable': 0,
 'BLEU': 26.292006542027888,
 'Corpus quality estimation': {'Compression ratio': 0.6523754417796537,
  'Sentence splits': 1.2361866590323503,
  'Levenshtein similarity': 0.6440657390446873,
  'Exact copies': 0.006012024048096192,
  'Additions proportion': 0.17086825920411697,
  'Deletions proportion': 0.4993194558788454},
 'sim_score': 0.8623162187889726,
 'bertscore': {'precision': 0.8777497410774231,
  'recall': 0.9151053428649902,
  'f1': 0.8960384130477905}}

In [34]:
compute_metrics_for_gpt4(eval)

499


{'Corpus quality estimation': {'Compression ratio': 0.6879483647413309,
  'Sentence splits': 1.2771972516461492,
  'Levenshtein similarity': 0.6119874455328814,
  'Exact copies': 0.0,
  'Additions proportion': 0.24135016777113252,
  'Deletions proportion': 0.5323608648712668},
 'sim_score': 0.8619006387217489}

In [35]:
corpus = eval + train

In [36]:
compute_metrics_for_gpt4(data=corpus)

9628


{'Corpus quality estimation': {'Compression ratio': 0.6732875568246265,
  'Sentence splits': 1.3241796001740958,
  'Levenshtein similarity': 0.5980057872325162,
  'Exact copies': 0.0002077274615704196,
  'Additions proportion': 0.24729608997466082,
  'Deletions proportion': 0.5526963892275472},
 'sim_score': 0.8620795755990324}